In [1]:
# Install required packages
!pip install -q transformers torch transformers_stream_generator optimum auto-gptq tiktoken

In [2]:
# Import necessary libraries
import io
import pandas as pd
import transformers
import torch
import optimum
# from google.colab import files

In [3]:
# Read the CSV files into DataFrames
forecast_arima = pd.read_csv('/content/drive/MyDrive/AAAIHack/bond_yields_predicted_GAN.csv')
forecast_garima = pd.read_csv('/content/drive/MyDrive/AAAIHack/bond_yields_predicted_actual.csv')
past_year_data = pd.read_csv('/content/drive/MyDrive/AAAIHack/bond_yields_previous_FY.csv')

In [4]:
# Convert the DataFrames to CSV formatted strings (without the index)
arima_str = forecast_arima.to_csv(index=False)
garima_str = forecast_garima.to_csv(index=False)
past_year_str = past_year_data.to_csv(index=False)

In [5]:
user_prompt = f"""
You are a senior financial advisor specializing in fixed income and bond markets, with over 20 years of experience. Your expertise covers analyzing market trends, forecasting yield movements, and evaluating risk factors.

Using the data provided below:
1. Forecasted bond yields for the next 3 months from two ARIMA models based off different datas
2. Historical bond yield data for the past 12 months

Please analyze these details thoroughly and offer clear investment recommendations for each of the four bonds (AAA, BAA, US10Y, and Junk). For each bond, consider:
- Analysis of the forecasted trends
- Comparison with historical performance
- Consideration of yield volatility and risk-return trade-offs

Provide your output in the following structured format:
-----------------------------------------------------------
Bond: [Bond Name]
Recommendation: [BUY/HOLD/SELL]
Timing: [Specify timing, e.g., "BUY NOW", "BUY THIS MONTH", "BUY NEXT MONTH", "HOLD for 6 months", "SELL within 1 month"]
Risk: [Specify risk, e.g., "LOW", "MEDIUM", "HIGH"]
-----------------------------------------------------------
Ensure that your final output strictly follows this format for each bond.


Forecasted Bond Yields for the Next 3 Months:
--------------------------------------------
ARIMA Forecast:
{arima_str}

GARIMA Forecast:
{garima_str}

Historical Bond Yield Data (Past 12 Months):
--------------------------------------------
{past_year_str}

Please provide your analysis and recommendations in the structured format described.
"""

In [6]:
# # -----------------------------
# # Step 3: Set Up the LLM Pipeline
# # -----------------------------
# model_id = "WiroAI/WiroAI-Finance-Qwen-7B"

# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model_id,
#     model_kwargs={"torch_dtype": torch.bfloat16},
#     device_map="auto",
# )

# pipeline.model.eval()

# # Create the conversation messages.
# messages = [
#     {"role": "system", "content": "You are a Senior Financial Advisor with deep expertise in fixed income investments and bond market analysis."},
#     {"role": "user", "content": user_prompt},
# ]

# # Define terminators to mark the end of the generated text.
# terminators = [
#     pipeline.tokenizer.eos_token_id,
#     pipeline.tokenizer.convert_tokens_to_ids("<｜end▁of▁sentence｜>")
# ]

In [7]:
# # -----------------------------
# # Step 4: Generate Investment Advice
# # -----------------------------
# outputs = pipeline(
#     messages,
#     max_new_tokens=512,
#     eos_token_id=terminators,
#     do_sample=True,
#     temperature=0.9,
# )

# # Print the generated investment advice.
# # (Note: The output structure is based on the sample usage.)
# print(outputs[0]["generated_text"][-1]['content'])

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load the new, faster model from TheBloke/Qwen-7B-Chat-GPTQ
model_name_or_path = "TheBloke/Qwen-7B-Chat-GPTQ"  # Optionally add ":gptq-4bit-32g-actorder_True" for a specific branch
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    device_map="auto",
    trust_remote_code=True,
    revision="main"  # Change revision if needed
)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at TheBloke/Qwen-7B-Chat-GPTQ were not used when initializing QWenLMHeadModel: ['transformer.h.31.mlp.c_proj.bias', 'transformer.h.5.attn.c_proj.bias', 'transformer.h.10.attn.c_proj.bias', 'transformer.h.24.attn.c_proj.bias', 'transformer.h.29.mlp.c_proj.bias', 'transformer.h.12.mlp.c_proj.bias', 'transformer.h.24.mlp.w2.bias', 'transformer.h.4.attn.c_proj.bias', 'transformer.h.6.mlp.w2.bias', 'transformer.h.10.mlp.w2.bias', 'transformer.h.14.mlp.c_proj.bias', 'transformer.h.20.mlp.c_proj.bias', 'transformer.h.8.mlp.w1.bias', 'transformer.h.31.attn.c_proj.bias', 'transformer.h.8.mlp.c_proj.bias', 'transformer.h.10.mlp.c_proj.bias', 'transformer.h.16.mlp.c_proj.bias', 'transformer.h.30.mlp.w2.bias', 'transformer.h.7.mlp.c_proj.bias', 'transformer.h.22.mlp.w2.bias', 'transformer.h.26.mlp.w1.bias', 'transformer.h.13.attn.c_proj.bias', 'transformer.h.15.mlp.w2.bias', 'transformer.h.17.mlp.c_proj.bias', 'transformer.h.5.mlp.w2.bias', 'transformer.h.19.ml

The repository for TheBloke/Qwen-7B-Chat-GPTQ contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/TheBloke/Qwen-7B-Chat-GPTQ.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [ ]:
# Build the prompt template using the required tokens for this model
system_message = "You are a Senior Financial Advisor with deep expertise in fixed income investments and bond market analysis."
prompt_template = f"""<|im_start|>system
{system_message}<|im_end|>
<|im_start|>user
{user_prompt}<|im_end|>
<|im_start|>assistant
"""

# Create the text-generation pipeline with updated parameters
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

# Generate the investment advice using the new model
output = pipe(prompt_template)[0]['generated_text']
print(output)


Device set to use cuda:0
